#### DATABASE CONNECTION
https://docs.timescale.com/getting-started/latest/services/
<br>
After initiating a Timescale service we proceed to create a database connection using psycopg2.
<br>
There are two types of Timescale services. A “time-series service” is selected for this project because, unlike “Dynamic PostgreSQL”, it’s the one which includes the requested TimescaleDB. Both services are 100% PostgreSQL and use usage-based storage.

In [1]:
# PostgreSQL database adapter -> https://www.psycopg.org/docs/
import psycopg2

In [15]:
# DB connection, Expires on Jan 7th 2024
# Config data in timescale_credentials.sql file.  
conn = psycopg2.connect(
    host="fldr9dd2pe.ofb7ig8qpo.tsdb.cloud.timescale.com",
    port=39569,
    database="tsdb",
    user="tsdbadmin",
    password="pq8ornq2cnpjnm3p"
)
cursor = conn.cursor()

#### OPTIMIZATION AND SCALING CONSIDERATIONS

A) We could add an INDEX to the most populated, and ever growing table, "stock". This is a classic SQL approach to seek performance. 
<br>
The expression would be:

In [10]:
cursor.execute('''
    CREATE INDEX ix_stock ON test_db.stock(symbol, time DESC);
''')
# commit changes to the database to make changes persistent
conn.commit()
# close cursor to end session and release resources.
cursor.close()

B) Improve performance by using Hypertables for the time-series tables. In this case, it was for the stock table. This makes use of the "chunk" concept, the cornerstone of Timescale. The default segmente of time in Timescale is 7 days. 

C) Timescale also offers a compression functionalty. Here a quote:
<br>
<i>
Compressing your time-series data allows you to reduce your chunk size by more than 90%. This saves on storage costs, and keeps your queries operating at lightning speed.
<br>
When you enable compression, the data in your hypertable is compressed chunk by chunk. When the chunk is compressed, multiple records are grouped into a single row. The columns of this row hold an array-like structure that stores all the data. This means that instead of using lots of rows to store the data, it stores the same data in a single row. Because a single row takes up less disk space than many rows, it decreases the amount of disk space required, and can also speed up your queries.
</i>
<br><br>
To take into consideration: Most indexes set on the hypertable are removed or ignored when reading from compressed chunks. Timescale creates and uses custom indexes to incorporate the segmentby and orderby parameters during compression.
<br><br>
https://docs.timescale.com/use-timescale/latest/compression/
<br>
https://www.youtube.com/watch?v=NOpRPzPL1x0